First step to perform a vector search is creating a vector policy for a container.
Vector Policies has the basic information database engine needs about the vector types and the metric used to compute distance/similarities.
Vector Policies has the basic information database engine needs about the vector data types, location, and the metric used to compute distance/similarities.
You can define multiple vectors in a container.

A Vector policy contains the following options :

"path" : It is required. Defines the location of vector.
"datatype" : float32 is the default value. int8 and uint8 are the other options.
"dimensions" : length of each vector in the path. All vectors should have the same length.
"distanceFunction" : The metrics used to compute distance/similarity.
   followings are supported for distanceFunction:
   "cosine"  
   "dot product"
   "euclidean"

You can use the Azure Portal or SDK to create a policy.


After creating a vector policy, we need to create a vector index on the property that will have vector.
Vector Indexes increase the efficiency and make your vector searches more affordable.
You can create multiple vector indexes.
"path" and "type" fields are required to create a vector index.

Three types of vector indexes are supported.

"flat" : Stores vectors on the same index as other props. Max length of a vector is 505
"quantizedFlat" : Compresses vectors before storing. Searches should have lower latency, higher throughput, and lower RU cost comparing to "flat"
"diskANN" : Deveoped by Microsoft. It offers a better performance and cost if you have more than 50k vectors in a physical partition.

"vectorIndexes":[
    {
        "path": "/vectorloc",
        "type": "diskANN"
    }
]

In [3]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Collections.ObjectModel;

Installed Packages Microsoft.Azure.Cosmos, 3.51.0 Newtonsoft.Json, 13.0.3

In [4]:
var cstring = "AccountEndpoint=https://localhost:8081/;AccountKey=C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(cstring);
var db = client.GetDatabase("StackOverflow");
 
List<Embedding> embeddings = new List<Embedding>()
{
      new Embedding()
      {
          Path = "/bodyvector",
          DataType = VectorDataType.Float32,
          DistanceFunction = DistanceFunction.Cosine,
          Dimensions = 1536,
      }
};
var collection = new Collection<Embedding>(embeddings);

ContainerProperties props = new ContainerProperties("VectorPosts2", "/OwnerUserId"){    
    VectorEmbeddingPolicy = new (collection),
    IndexingPolicy = new IndexingPolicy(){
        VectorIndexes = new Collection<VectorIndexPath>()
        {
            new VectorIndexPath()
            {
                Path = "/bodyvector",
                Type = VectorIndexType.QuantizedFlat
            }
        }
    }
};
props.IndexingPolicy.IncludedPaths.Add(new IncludedPath()
{
    Path = "/*"
});
props.IndexingPolicy.ExcludedPaths.Add(new ExcludedPath()
{
    Path = "/bodyvector/?"
});

var postContainer = await db.CreateContainerIfNotExistsAsync(props, throughput: 4000);

/bodyvector property will be used store vectors.
Code creates a vector index for /bodyvector property. 
We don't want to generate regular indexes on /bodyvector values. 
That's why code adds /bodyvector to the ExcludedPaths in Indexing Policy.
After the code completes succesfully, open your Azure Cosmos Db Emulator and find the new container.
Then click on Scale & Settings menu option and find the Container Vector Policy tab to see the details.
Next, we will generate vectors to store in /bodyvector